In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
import time
import re
import shutil
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data.sampler import SequentialSampler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score, jaccard_score, f1_score, homogeneity_completeness_v_measure, adjusted_rand_score
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score, pairwise_distances
import numpy as np
from scipy.optimize import linear_sum_assignment
import torch.nn.functional as F

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!wget -O Stsbenchmark.tar.gz http://ixa2.si.ehu.es/stswiki/images/4/48/Stsbenchmark.tar.gz
shutil.unpack_archive('./Stsbenchmark.tar.gz', extract_dir='./', format='gztar')

--2024-01-04 05:00:12--  http://ixa2.si.ehu.es/stswiki/images/4/48/Stsbenchmark.tar.gz
Resolving ixa2.si.ehu.es (ixa2.si.ehu.es)... 158.227.106.100
Connecting to ixa2.si.ehu.es (ixa2.si.ehu.es)|158.227.106.100|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://ixa2.si.ehu.eus/stswiki/images/4/48/Stsbenchmark.tar.gz [following]
--2024-01-04 05:00:28--  http://ixa2.si.ehu.eus/stswiki/images/4/48/Stsbenchmark.tar.gz
Resolving ixa2.si.ehu.eus (ixa2.si.ehu.eus)... 158.227.106.100
Connecting to ixa2.si.ehu.eus (ixa2.si.ehu.eus)|158.227.106.100|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 409630 (400K) [application/x-gzip]
Saving to: ‘Stsbenchmark.tar.gz’

Stsbenchmark.tar.gz 100%[===================>] 400.03K   387KB/s    in 1.0s    

2024-01-04 05:01:33 (387 KB/s) - ‘Stsbenchmark.tar.gz’ saved [409630/409630]



In [4]:
def getSTSBenchmarkSents(filename='sts-train.csv', root='stsbenchmark/', encoding='utf-8'):
  f = open(root+filename, 'r', encoding=encoding)
  s1, s2, target = [], [], []
  for line in f:
    example = re.split(r'\t+', line)
    if len(example) > 7:
      example = example[:-2]
    s2.append(example[-1])
    s1.append(example[-2])
    target.append(float(example[-3]))
  print("{} samples: {}".format(filename, len(target)))
  return s1, s2, target

In [5]:
s1_test,s2_test,target_test= getSTSBenchmarkSents(filename='sts-test.csv')

sts-test.csv samples: 1379


In [6]:
BERT_PATH = "bert-base-uncased"
root_drive = '/content/drive/MyDrive/Tesis/STS_Benchmark/transformer_tunned_BERT/uncase_base/'

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {device}')

Device: cuda


In [12]:
#(num_sent, num_capa, num_cabezal)
#all_attentions_matrix = torch.load(root_drive + BERT_PATH + '_all_attentions_test_complete.pth')
all_attentions_matrix = torch.load(root_drive + BERT_PATH + '_all_attentions_test_STS-B.pth')

In [13]:
layers = 12
heads = 12
num_sentences = list(all_attentions_matrix.keys())[-1][0]+1
attentions_concat_heads = {}
attentions_matrix_list = []
attentions_list = []

for i in range(num_sentences):
  for j in range(layers):
    tensor_list = []
    for k in range(heads):
      tensor_list.append(torch.tensor(all_attentions_matrix[(i,j,k)]['vectors']).flatten())
    #attentions_concat_heads[(i,j)] = torch.stack(tensor_list).unsqueeze(0).permute(0, 2, 1)
    stack = torch.stack(tensor_list)
    attentions_concat_heads[(i,j)] = stack
    attentions_matrix_list.append(stack)
    #attentions_list.append(((i,j),all_attentions_matrix[(i,j,k)]['label'], stack))
    attentions_list.append(((i,j),all_attentions_matrix[(i,j,k)]['sequence'],all_attentions_matrix[(i,j,k)]['label'],all_attentions_matrix[(i,j,k)]['dimension'],stack))
attentions_list = [(id,s,label,dim,tensor.permute(1,0)) for id, s, label, dim, tensor in attentions_list]

In [14]:
attentions_list[0]

((0, 0),
 's1: A girl is styling her hair. s2: A girl is brushing her hair.',
 2.5,
 17,
 tensor([[0.0482, 0.7595, 0.7518,  ..., 0.0573, 0.6803, 0.8649],
         [0.1027, 0.0157, 0.0206,  ..., 0.0765, 0.0337, 0.0415],
         [0.0210, 0.0078, 0.0071,  ..., 0.0599, 0.0072, 0.0022],
         ...,
         [0.0825, 0.0444, 0.0269,  ..., 0.0435, 0.0954, 0.0390],
         [0.1232, 0.3039, 0.3509,  ..., 0.0428, 0.1945, 0.0027],
         [0.2252, 0.0845, 0.1228,  ..., 0.0388, 0.3616, 0.2844]]))

In [15]:
attentions_list[0][4].shape

torch.Size([289, 12])

Dataloader

In [16]:
# Crear un sampler secuencial
# attentions_list = dataset
# SequentialSampler does not perform any shuffling or random selection of items.
sampler = SequentialSampler(attentions_list)

# Definir el tamaño del lote
batch_size = 12 # always 12, because it is the number of attention layers, 12 layers for the same sentence

# Crear el DataLoader sin un BatchSampler
dataloader = DataLoader(attentions_list, batch_size=batch_size, sampler=sampler)

In [17]:
len(dataloader) #1379 OK because there are 1379 sentences, 1 batch = 1 same sentence , 12 elements in the batch because there are 12 layers

1379

In [18]:
sr = next(iter(dataloader))
sr[0] # ids (num_sent, num_layer)
sr[1] # similarity label
sr[2].shape
sr[3].shape
sr[4].shape #[batch_size, len_sentence, input_size] with attentions ([12,289,12])

torch.Size([12, 289, 12])

In [19]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [20]:
def train_loop(model, iterator, optimizer, criterion, device=device, clip = 1.0):
    #Training loop
    model.train()
    loss_sum = 0
    seed = 42
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    #torch.cuda.set_device(0)

    for i, (_,_,_,_,input) in enumerate(iterator):
        optimizer.zero_grad()
        output, _ = model(input)
        loss = criterion(output, input)
        loss.backward()
        #prevent gradients from exploding
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        #Update params
        optimizer.step()
        loss_sum += loss.item()

    epoch_train_loss = loss_sum * batch_size / len(iterator)

    return epoch_train_loss

In [21]:
# Extraer el vector latente fijo de cada elemento del batch
def extract_latent_vectors(model, dataloader, device, model_type='RNN'):
    model.eval()
    vector_representations = {}

    with torch.no_grad():
        for (id,s,label,dim,input) in dataloader:
            latent_vectors = []
            if model_type == 'RNN':
              _, (latent_representation, _) = model.encoder(input)
              latent = latent_representation.squeeze(0)
            else:
              latent_representation = model.encoder(input)
              latent = latent_representation.squeeze(0).squeeze(1)
            tuples = list(zip(id[0].tolist(), id[1].tolist()))
            #latent = latent_representation.squeeze(0)
            for i in range(latent.size(0)):
              latent_vectors.append(latent[i].numpy())
              vector_representations[tuples[i]] = { 'vector' : latent[i].numpy(), 'sequence': s, 'label': label, 'dimension':dim}

    return vector_representations

In [22]:
# Definir la arquitectura del autoencoder con LSTM
class AutoencoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, latent_size = 128):
        super(AutoencoderLSTM, self).__init__()
        self.encoder = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.decoder = nn.LSTM(hidden_size, input_size, batch_first=True)
    def forward(self, x):
        # Codificación
        #o = [1, 12, 128] = [batch_size, len_sents, hidden_size]
        #x = [1, 12, 289] = [batch_size, len_sents, input_size]
        #print(x.shape)
        o, (h_n, _) = self.encoder(x)
        #h = [1, 1, 128] = [batch_size, num_layers * num_directions, hidden_size]
        # Reducción a tamaño latente
        #latent = [1, 128] = [num_layers * num_directions, hidden_size]
        latent = h_n.squeeze(0)
        # Decodificación
        #output, _ = self.decoder(latent.unsqueeze(0).repeat(1, x.size(1), 1))
        output, _ = self.decoder(o)
        return output, latent


In [23]:
NUM_EPOCHS = 4
best_valid_loss = float('inf')
model_name = 'Autoencoder_LSTM'
train_loss_values = []
history = {"train": {"loss": []}}

input_size = 12  #due to 768 dimensión of BERT
hidden_size = 2  # size of fixed vector #laten dim
learning_rate = 0.001 #0.00025 #0.0007

seed = 42
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)


model = AutoencoderLSTM(input_size, hidden_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(NUM_EPOCHS):

    start_time = time.time()
    epoch_train_loss = train_loop(model,dataloader,optimizer,criterion,device)
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    train_loss_values.append(epoch_train_loss)

    history["train"]["loss"].append(epoch_train_loss)

    print('-' * 80)
    #print(f'Epoch: {epoch+1:03}/{NUM_EPOCHS} | Epoch Time: {epoch_mins}m {epoch_secs}s | Train loss: {epoch_train_loss:.4f} | Train acc: {epoch_train_acc:.4f} | Dev loss: {epoch_dev_loss:.4f} | Dev acc: {epoch_dev_acc:.4f}')
    print(f'Epoch: {epoch+1:03}/{NUM_EPOCHS} | Epoch Time: {epoch_mins}m {epoch_secs}s | Train loss: {epoch_train_loss:.4f}')


--------------------------------------------------------------------------------
Epoch: 001/4 | Epoch Time: 0m 45s | Train loss: 0.0884
--------------------------------------------------------------------------------
Epoch: 002/4 | Epoch Time: 0m 45s | Train loss: 0.0511
--------------------------------------------------------------------------------
Epoch: 003/4 | Epoch Time: 0m 45s | Train loss: 0.0472
--------------------------------------------------------------------------------
Epoch: 004/4 | Epoch Time: 0m 44s | Train loss: 0.0466


In [24]:
vector_representations = extract_latent_vectors(model, dataloader, device)

In [25]:
vector_representations[(0,0)]['vector']

array([ 0.08917949, -0.19299325], dtype=float32)

In [26]:
# Guardar el diccionario en un archivo
torch.save(vector_representations, root_drive + BERT_PATH + str(hidden_size) + '_vector_representations_ATTENTIONS_2D.pth')

In [27]:
hidden_size = 2  # size of fixed vector
vector_representations = torch.load(root_drive + BERT_PATH + str(hidden_size) + '_vector_representations_ATTENTIONS_2D.pth')
# el archivo '_vector_representations_ATTENTIONS_2D.pth' puede ser directamente cargado en la herramienta visual de interpretabilidad opción ()

In [28]:
len(vector_representations) #(num_sentence,layer)

16548

In [29]:
def get_representations_per_layer(num_sentences, vector_representations, layers = 12):
  vectors_per_layer = {}
  labels = {}
  for l in range(layers):
    #vectors_per_layer[l] = np.array([vector_representations[(i,l)]['vector'].detach().numpy() for i in range(num_sentences)])
    vectors_per_layer[l] = np.array([vector_representations[(i,l)]['vector'] for i in range(num_sentences)])
  labels = { i: vector_representations[(i,0)]['label'][0].item() for i in range(num_sentences)}
  sequences = { i: vector_representations[(i,0)]['sequence'][0] for i in range(num_sentences)}
  dimensions = { i: vector_representations[(i,0)]['dimension'][0].item() for i in range(num_sentences)}
  return vectors_per_layer, sequences, labels, dimensions

In [30]:
vectors_per_layer, sequences, labels, dimensions = get_representations_per_layer(num_sentences, vector_representations)

In [31]:
# Guardar el diccionario en un archivo
torch.save(vectors_per_layer, root_drive + BERT_PATH + str(hidden_size) + '_vectors_per_layer_ATT_LSTM_prueba.pth')